# Home Assignment 2 (26 pts)

Submit your solution via Ilias until 23.59h on Friday, October 25th. Any later submission is not possible.

Submit your solutions in teams of 4 students. Unless explicitly agreed otherwise in advance, submissions from teams with more or less members will NOT be graded (i.e., count as failed).

**Make sure that all team members are part of the submitting group on Ilias.**

You may use the code from the exercises and basic functionalities that are explained in the official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members may be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points.
* If you use packages that are not available on the default or conda-forge channel, list them below. Also add a link to installation instructions. 
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Avoid reusing variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post in the student forum in Ilias, so we can clear up such questions for all students in the course.

Additional packages (if any):
 - Example: `powerlaw`, https://github.com/jeffalstott/powerlaw

In [18]:
from typing import List, Union, Dict, Set, Tuple, Sequence

### Task 1: WordNet word similarity (9 points)

In this task, we want to implement the path-based similarity between two words in WordNet (https://www.nltk.org/api/nltk.corpus.reader.wordnet.html) using the NLTK package. The path-based word similarity between two words is given by
$$
\frac{1}{1+d}
$$
where $d$ is the distance of the shortest path in the hypernym/hyponym hierarchy tree in WordNet between any pair of synsets that are associated with the two input words.

From NLTK you should __only__ use the `synsets`, `hypernyms` and `instance_hpyernyms` functions.

The following subtasks build on each other, i.e. the functions of the preceding subtasks can be used for the current subtask.

_Note: here we define the distance of a synset to itself as 0, the distance to a direct hypernym as 1, ..._

In [19]:
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset

__a)__ Write a function ``shortest_paths_to`` that takes a synset as input and computes the shortest paths to all nodes on the way to the root in the hypernym hierarchy tree of WordNet. The function should return a dictionary that matches all visited hypernyms on the way(s) to the root to the distance of the shortest path from the input synset. Consider that a synset might have multiple paths to the root and that some nodes might appear in multiple paths. However, we only want to store the shortest distances. Moreover, keep in mind that the input synset might be an instance. __(3 pts)__

Use the signature in the cell below.

__Example:__ _Calling_ ``shortest_paths_to(s)`` _on the synset_ ``s = wn.synset('calculator.n.01')`` _should yield the following result:_

``
{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 1,
 Synset('person.n.01'): 2,
 Synset('causal_agent.n.01'): 3,
 Synset('organism.n.01'): 3,
 Synset('physical_entity.n.01'): 4,
 Synset('living_thing.n.01'): 4,
 Synset('entity.n.01'): 5,
 Synset('whole.n.02'): 5,
 Synset('object.n.01'): 6}
``

In [20]:
def shortest_paths_to(start_syn: Synset) -> Dict[Synset, int]:
    """Compute the shortest distance to all nodes on paths to the root.
    :param start_syn: synset to which we want to compute the shortest distances
    :return: dict that matches all visited hypernyms to their distance to the input synset  
    """ 
    list: List[Tuple[Synset, int]] = [(start_syn, 0)]  
    
   
    shortest_paths = {start_syn: 0}
    
    while list:
        
        synset, distance = list.pop(0)
        
        hypernyms = synset.hypernyms() + synset.instance_hypernyms()
        
        for hypernym in hypernyms:
            distance_hypernym = distance + 1
   
            if hypernym not in shortest_paths or distance_hypernym < shortest_paths[hypernym]:
                shortest_paths[hypernym] = distance_hypernym
                list.append((hypernym, distance_hypernym))
    
    return shortest_paths
    

In [21]:
s = wn.synset('calculator.n.01')
shortest_paths_to(s)

{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 1,
 Synset('person.n.01'): 2,
 Synset('organism.n.01'): 3,
 Synset('causal_agent.n.01'): 3,
 Synset('living_thing.n.01'): 4,
 Synset('physical_entity.n.01'): 4,
 Synset('whole.n.02'): 5,
 Synset('entity.n.01'): 5,
 Synset('object.n.01'): 6}

__b)__ Write a function ``merge_paths`` that gets two dictionaries that map synsets to shortest distances (you can assume they were created by the function from __a)__) and merges them. The function shold return a dictionary that includes all synsets and distances that appear in any of the input dictionaries. If a synset appears in both input dictionaries, we want to keep the shorter distance. Leave the input dictionaries unaltered. __(1.5 pts)__

Use the signature in the cell below.

In [22]:
def merge_paths(p1: Dict[Synset, int], p2: Dict[Synset, int]) -> Dict[Synset, int]:
    """Merge two paths keeping the shorter distance for synsets that appear more than once.
    :param p1: first dict that maps synsets to their shortest distances
    :param p2: second dict that maps synsets to their shortest distances
    :return: merged dict
    """
    result = p1.copy()
    
    for synset, distance in p2.items():
        
        if synset in result:
            result[synset] = min(result[synset], distance)
        else:
            result[synset] = distance
    
    return result
    

In [23]:
s = wn.synset('calculator.n.01')
p = wn.synset('expert.n.01')

merge_paths(shortest_paths_to(s), shortest_paths_to(p))

{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 0,
 Synset('person.n.01'): 1,
 Synset('organism.n.01'): 2,
 Synset('causal_agent.n.01'): 2,
 Synset('living_thing.n.01'): 3,
 Synset('physical_entity.n.01'): 3,
 Synset('whole.n.02'): 4,
 Synset('entity.n.01'): 4,
 Synset('object.n.01'): 5}

__c)__ Write a function ``all_hypernym_paths`` that gets a word as input and returns a dictionary that maps all hypernyms that are reachable from the set of synsets associated with the word to the shortest distance leading there. __(1.5 pts)__

Use the signature in the cell below.

In [24]:
def all_hypernym_paths(word: str) -> Dict[Synset, int]:
    """Get all hypernyms of all synsets associated with the input word and compute the shortest distance leading there.
    :param word: input word
    :return: dict that matches all reachable hypernyms to their shortest distance 
    """
    all_paths: Dict[Synset, int] = {}
    
    synsets = wn.synsets(word)
    
    for synset in synsets:
        paths = shortest_paths_to(synset)
        
        all_paths = merge_paths(all_paths, paths)
    
    return all_paths

In [25]:
all_hypernym_paths('calculator')

{Synset('calculator.n.01'): 0,
 Synset('expert.n.01'): 1,
 Synset('person.n.01'): 2,
 Synset('organism.n.01'): 3,
 Synset('causal_agent.n.01'): 3,
 Synset('living_thing.n.01'): 4,
 Synset('physical_entity.n.01'): 4,
 Synset('whole.n.02'): 5,
 Synset('entity.n.01'): 5,
 Synset('object.n.01'): 6,
 Synset('calculator.n.02'): 0,
 Synset('machine.n.01'): 1,
 Synset('device.n.01'): 2,
 Synset('instrumentality.n.03'): 3,
 Synset('artifact.n.01'): 4}

__d)__  Write a function ``get_dist`` that returns the word similarity between two input words, according to the formula given in the task description at the beginning.  __(3 pts)__

Use the signature in the cell below.

In [26]:
def get_dist(w1 : str, w2 : str) -> float:
    """Compute the similarity between two input words in the WordNet hierarchy tree.
    :param w1: first input word
    :param w2: second input word
    :return: word similarity
    """
   
    paths_w1 = all_hypernym_paths(w1)
    paths_w2 = all_hypernym_paths(w2)
    
    d = None # minimum distance between two words
    
    
    for synset1, dist1 in paths_w1.items():
        if synset1 in paths_w2:
           
            dist2 = paths_w2[synset1]
            total_dist = dist1 + dist2
           
            if d is None:
                d = total_dist
            if d < total_dist:
                d = total_dist

    
    # no similarity
    if d is None:
        return 0.0
    
    word_similarity = 1 / (1 + d)
    return word_similarity

In [27]:
get_dist('calculator', 'expert')

0.08333333333333333

### Task 2: Lesk algorithm (4 points)

In this task we want to implement a simple version of the Lesk algorithm, a thesaurus-based method for word sense disambiguation. Given a target word $w$ and a context, the algorithm finds the word sense that is most fitting in the context. To achieve this, the Lesk algorithm computes the number of overlapping words between the context sentence and the definitions of the WordNet synsets, associated with $w$.

Write a function ``lesk`` that takes a word and a context string (e.g. a sentence) and returns the most fitting sense from the synsets associated with the word and the corresponding context overlap. The most fitting sense is the one whose definition shares the most words with the context string. Before matching tokens, make sure to 

* only include valid tokens (cf. HA 1, task 2a)
* remove stopwords
* only match stems of words (e.g. consider the ``PorterStemmer`` from ``nltk``)

When computing the context overlap, count each stemmed word only once, even if they occur multiple times. If there is no fitting synset, i.e., the context overlap between the context and the synset definitions is 0, return None instead.

Use the signature in the cell below.

In [28]:
# HA 1, task 2a)
from nltk.corpus.reader.util import StreamBackedCorpusView 
from nltk.corpus import stopwords
import re
import string

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    :param tokens: list of tokens that should be cleaned
    :param remove_stopwords: bool indicating if stopwords should be removed 
                             False by default
    :return: list of valid tokens
    """
    valid = []
    punct = string.punctuation
    stop = stopwords.words('english')
    digit = re.compile(r"\d+")
    
    for t in tokens:
        if t in punct:
            continue
        if remove_stopwords and t.lower() in stop:
            continue
        if re.fullmatch(digit, t):
            continue
        valid.append(t.lower())
    return valid

In [29]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def lesk(word: str, context: str) -> (Synset, int):
    '''
    Compute the most probable sense of a word in the given context.
    :param word: ambiguous word
    :param context: context in which the word appears
    :returns: 
        - synset with the most likely word sense
        - context overlap of synset and context          
    '''
    # your code here

    # Get all synsets for the given word
    synsets = wn.synsets(word)
    if not synsets:
        return None

    # Tokenize and clean the context
    valid_context = get_valid_tokens(context.split(), remove_stopwords=True)
    valid_context_set = set(stemmer.stem(t) for t in valid_context)  # Stem context tokens
    
    max_overlap = 0
    best_synset = None

    for synset in synsets:
        # Get definition of the current synset
        definition = synset.definition()
        # Tokenize and stem the definition
        definition_tokens = get_valid_tokens(definition.split(), remove_stopwords=False)
        definition_stems = set(stemmer.stem(t) for t in definition_tokens)

        # Calculate overlap with the valid context
        overlap = len(valid_context_set.intersection(definition_stems))
        if overlap > max_overlap:
            max_overlap = overlap
            best_synset = synset

    # Return the best synset and the overlap count
    return (best_synset.name(), max_overlap) if best_synset else None

### Task 3: Markov chains (13 points)

In this task we want to create a language model by using the independence assumption af Markov. We therefore assume that the probability of a word is only dependent on a fixed number of preceding words. E.g. by restricting the number of preceding words to $n$ we can approximate the probability of a word $w_{i}$ following a sequence of words $w_1, ..., w_{i-1}$ by:

$P(w_{i}|w_1, ..., w_{i-1}) \approx P(w_{i}|w_{i-n}, ..., w_{i-1})$

We will first train our model on a given corpus and then use it to automatically generate text.

Throughout this task we will define a single class with different functions. If you're unsure how to access class methods and attributes, take a look at the documentation (https://docs.python.org/3/tutorial/classes.html).

__a) Collecting the counts (3 pts)__

Write a function `process_corpus` that takes a corpus of text (as a sequence of tokens) as input and counts how often an n-gram of length $n$ (``context_len=n``) is followed by a certain word (the n-grams should __not__ be padded). The function should return a dictionary that maps every n-gram that is observed in the corpus to an inner dictionary. The inner dictionary maps each word to a number, that indicates how often the word succeeds the n-gram in the given corpus. We will need these counts to compute the conditional probabilities $P(w_{i}|w_{i-n}, ..., w_{i-1})$.
Additionally, also return the entire vocabulary $V$ (i.e. the set of all unique tokens that appear in the corpus).

Make sure your implementation is efficient by using e.g. ``Counter`` and ``defaultdict`` from the package ``collections``.   

__b) Conditional probabilities (3 pts)__

Write a function `transition_prob` that takes an n-gram $(w_{i-n}, ..., w_{i-1})$ and a word $w_{i}$ of the vocabulary $V$ as input and returns the conditional probability that the given n-gram is followed by the input word $w_{i}$. Recall that this conditional probability can be computed as follows:

$P(w_{i}|w_{i-n}, ..., w_{i-1}) = \frac{\text{Count}(w_{i-n}, ..., w_{i-1}, w_{i})}{\sum_{w \in V}\text{Count}(w_{i-n}, ..., w_{i-1}, w)}$

If the n-gram has never been observed, return $\frac{1}{|V|}$.

__c) Most likely word (3 pts)__

Write a function `most_likely_word` that gets an n-gram as input and returns the word that is most likely to succeed the given n-gram. In case there are multiple words that are equally likely to follow the given n-gram, return all of them. 
Note that you should **not** loop over the **entire** vocabulary to obtain the most likely word.
In case the given n-gram has never been observed, return the entire vocabulary.

__d) Generating text (2 pts)__

Write a function `generate_text` that generates a text sequence of length `k`, given a starting sequence of words (`ngram`). The function should choose the most likely next word in every step; in case there are multiple equally likely words, randomly choose one. You should return a list of ``k`` words, that includes the starting sequence and is the most probable continuation. 


Please do not implement other functions for the MarkovModel class.

Use the function signatures in the cell below.

In [30]:
from collections import defaultdict, Counter
import random
from nltk.util import ngrams

class MarkovModel():
    '''Markov model for generating text.'''
    
    def __init__(self, tokens: Sequence[str], context_len: int):
        '''
        :param tokens: text corpus on which the model is trained on as an iterator of tokens
        :param context_len: length of the n-gram (number of preceding words)
        '''
        self.context_len = context_len 
        self.counts, self.v = self.process_corpus(tokens)
        
    def process_corpus(self, tokens: Sequence[str]) -> (Dict[Tuple[str, ...], Dict[str, int]], Set):
        '''Training method of the model, counts the occurences of each word after each observed n-gram.
        :param tokens: text corpus on which the model is trained on as an iterator of tokens
        :returns: 
            - nested dict that maps each n-gram to the counts of the words succeeding it
            - the whole vocabulary as a set
        '''
        ngram_dict = defaultdict(Counter)  
        vocabulary = set(tokens) 

       
        for i in range(len(tokens) - self.context_len):
            ngram = tuple(tokens[i:i + self.context_len])  
            next_word = tokens[i + self.context_len]  
            ngram_dict[ngram][next_word] += 1  
        
        return ngram_dict, vocabulary
        
        
    def transition_prob(self, ngram: Tuple[str, ...], word: str) -> float:
        '''Compute the conditional probability that the input word follows the given n-gram.
        :param ngram: string tuple that represents an n-gram
        :param word: input word
        :return: probability that the n-gram is followed by the input word
        '''
        
        if ngram not in self.counts:
            return 1 / len(self.v)
        
        
        
        word_count = self.counts[ngram][word]
        
        
        total_count = sum(self.counts[ngram].values())

        if(total_count > 0):
            return word_count / total_count
        else:
            return 1 / len(self.v)
        
    
    def most_likely_word(self, ngram: Tuple[str, ...]) -> Set[str]:
        '''Computes which word is most likely to follow a given n-gram.
        :param ngram: n-gram we are interested in
        return: set of words that are most likely to follow the n-gram
        '''
        
        if ngram not in self.counts:
            return list(self.v)
        
        
        following = self.counts[ngram]
        word_prob = {word: self.transition_prob(ngram, word) for word in following}
        max_prob = max(word_prob.values())
        most_likely = [word for word, prob in word_prob.items() if prob == max_prob]
        
        return most_likely
        
    
    def generate_text(self, ngram: Tuple[str, ...], k: int) -> List[str]:
        '''Generates a text sequence of length k, given a starting sequence.
        :param ngram: starting sequence
        :param k: total number of words in the generated sequence
        :return: sequence of generated words, including the starting sequence
        '''
        text = list(ngram)  
        
        
        for i in range(k - len(ngram)):
            
            next_words = self.most_likely_word(tuple(text[-self.context_len:]))
            
            
            next_word = random.choice(next_words)
            
            
            text.append(next_word)
        
        return text
        

__e) Apply the model to a corpus (2 pts)__

Finally, we want to apply our functions to the King James Bible (`'bible-kjv.txt'`) that is part of the ``gutenberg`` corpus. Use the function from HA 1, task 2a) to obtain a list of valid tokens (do not remove stopwords) from the King James Bible.

Initialize the MarkovModel with the list of valid tokens and ``context_len=3`` and answer the following subtasks:

i) What is the probability that the word ``babylon`` follows the sequence ``the king of``? 

ii) What are the most likely words to follow the sequence ``the world is``? 

iii) Generate a sequence of length 20 that starts with ``mother mary was``.


In [31]:
import nltk
import string
from nltk.corpus.reader.util import StreamBackedCorpusView
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from typing import List, Union, Dict, Set, Tuple

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    #:param tokens: list of tokens that should be cleaned
    #:param remove_stopwords: bool indicating if stopwords should be removed 
                             #False by default
    #:return: list of valid tokens
    #"""
    # your code here
    stop_words = set (stopwords.words('english')) if remove_stopwords else set()

    punctuation_set = set(string.punctuation)

    valid_tokens = []

    if isinstance(tokens, StreamBackedCorpusView):
        tokens = list(tokens)
    
    for token in tokens: 
        token = token.lower()
        if all(char in punctuation_set for char in token):
            continue
        
        if token.isdigit(): 
            continue

        if remove_stopwords and token.lower() in stop_words: 
            continue

        valid_tokens.append(token)

    return valid_tokens

In [32]:
nltk.download('gutenberg')
raw = gutenberg.raw('bible-kjv.txt')

words = gutenberg.words('bible-kjv.txt')

valid_tokens = get_valid_tokens(words, remove_stopwords=False)
markov_model = MarkovModel(valid_tokens, context_len=3)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/elisarogers/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [33]:
#i)
ngram = ('the', 'king', 'of')
word = 'babylon'
prob_babylon = markov_model.transition_prob(ngram, word)
print(f"Probability that 'babylon' follows 'the king of': {prob_babylon}")

Probability that 'babylon' follows 'the king of': 0.1906779661016949


In [34]:
#ii)
ngram = ('the', 'world', 'is')


most_likely = markov_model.most_likely_word(ngram)
print(f"Most likely words to follow 'the world is': {most_likely}")

Most likely words to follow 'the world is': ['mine', 'gone', 'crucified', 'enmity', 'the']


In [35]:
#iii)
ngram = ('mother', 'mary', 'was')

text = markov_model.generate_text(ngram, 20)
print(f"Generated sequence: {' '.join(text)}")

Generated sequence: mother mary was espoused to joseph before they came together she was found with child of gilead that they might


## Note to Professor: 
we additionally imported random module from python standard library